In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
API_KEY = '6cc211017f64141072497d1766550'
BASE_URL = 'https://api.meetup.com'

In [10]:
# get events
urlname = 'DE-ProD-SF'

events_url = BASE_URL + '/' + urlname + '/events'
params = {'sign':'true','key': API_KEY, 'status': 'past'}
response = requests.get(events_url, params = params)
events = response.json()

event_ids = []
event_desc_raw= []

for event in events:
    event_ids.append(event['id'])
    event_desc_raw.append(event['description'])

event_description = []
for i in xrange(len(event_ids)):
    soup = BeautifulSoup(event_desc_raw[i], 'html.parser')
    event_description.append(soup.get_text())
    
df_events= pd.DataFrame([event_ids, event_description]).T
df_events.columns = ['event_id', 'event_description']
df_events

,event_id,event_description
0,228968660,Exercise your critical thinking with a data en...
1,228648803,"*Note, expedite your check in at Galvanize; re..."


In [11]:
# get rsvps

member_id = []
event_id = []

for i in event_ids:
    rsvps_url = BASE_URL + '/' + urlname +'/events/' + i + '/rsvps'
    params = {'sign':'true','key': API_KEY}
    response = requests.get(rsvps_url, params = params)
    rsvps = response.json()
    
    for rsvp in rsvps:
        member_id.append(rsvp['member']['id'])
        event_id.append(rsvp['event']['id'])
               
df_rsvps = pd.DataFrame([member_id, event_id]).T
df_rsvps.columns = ['member_id', 'event_id']
df_rsvps

,member_id,event_id
0,87429312,228968660
1,149624662,228968660
2,129647262,228968660
3,132426572,228968660
4,67449572,228968660
5,111657512,228968660
6,198433955,228968660
7,11847383,228968660
8,10048695,228968660
9,14704591,228968660


In [12]:
# member + event
df = pd.merge(df_events, df_rsvps, on = 'event_id')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 0 to 37
Data columns (total 3 columns):
event_id             38 non-null object
event_description    38 non-null object
member_id            38 non-null object
dtypes: object(3)
memory usage: 1.2+ KB


In [16]:
df.to_json('data2.json')